In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Create Policy to Restrict Portfolio Updates in a Period

This notebook shows how to create a policy to restrict portfolio updates inbetween 2 effective dates by denying any write/delete activity to all portfolios in a given scope.

Attributes
----------
policies
portfolios
"""

toggle_code("Hide docstring")

# Create Policy to Restrict Portfolio Updates in a Period

It is sometimes necessary to lock a portfolio in a given time period and restrict users from being able to update/add/delete both transactions and holdings. A user may also want to lock a portfolio over a certain historical time horizon.

This notebook shows how to create a policy to restrict portfolio updates between 2 effective dates by denying write/delete activity to all portfolios in a given scope. 

We can then add this policy to a role and give this role to some or all users to lock down one or a series of portfolios in a specified time window.

![Timeline](img/timeline.JPG)

## Setup

In [2]:
# Import LUSID
import lusid
import finbourne_access

#import lusid_sample_data as import_data
from finbourne_access.utilities import ApiClientFactory as AccessApiClientFactory
from finbourne_access import models as AccessModels
from lusidjam import RefreshingToken
import fbnsdkutilities.utilities as utils

# Import Libraries
import os
import json
from IPython.display import Image

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
os.environ["FBN_LUSID_ACCESS_URL"] = access_api_url

access_api_factory = utils.ApiClientFactory(
    finbourne_access,
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define APIs
access_api = access_api_factory.build(finbourne_access.api.PoliciesApi)

In [4]:
# Define variables for use in the policy request body
scope = "ibor"
policy_code = "block-portfolio-updates"
policy_description = "Policy to block portfolio updates in a given date range"
effective_from = "2022-01-01T00:00:00.0000000+00:00"
effective_to = "2022-04-01T00:00:00.0000000+00:00"

## 1. Define and Create Policy

This policy denies any update, delete or add activity to portfolios in the given scope during the provided time frame. This prevents any transactions and holdings from being added, updated or deleted in this period.

This policy upon being created can be added to a role and applied to a user to add these restrictions to them (see 2. Next Steps).

In [5]:
request_body = AccessModels.PolicyCreationRequest(
    code=policy_code,
    description=policy_description,
    applications=["LUSID"],
    grant="Deny",
    selectors=[AccessModels.SelectorDefinition(
        id_selector_definition=AccessModels.IdSelectorDefinition(
            identifier={
                    "code": "*",
                    "scope": "ibor"
                },
            actions=[
                AccessModels.ActionId(
                    scope="default",
                    activity="Update",
                    entity="Portfolio",
                ),
                AccessModels.ActionId(
                    scope="default",
                    activity="Delete",
                    entity="Portfolio",
                ),
                AccessModels.ActionId(
                    scope="default",
                    activity="Add",
                    entity="Portfolio",
                )
            ]
        )
    )],
    _for=[AccessModels.ForSpec(
        effective_range = AccessModels.EffectiveRange(
            _from=effective_from,
            to=effective_to
        )
    )],
    when=AccessModels.WhenSpec(
        activate="0001-01-01T00:00:00.0000000+00:00",
        deactivate="9999-12-31T23:59:59.9999990+00:00"
    ),
)

In [6]:
# Create policy

# If the policy already exists an exception will be thrown
# To update a policy, delete it first and then re-run

try:
    response = access_api.create_policy(request_body)
    response
except finbourne_access.ApiException as e:
    print(json.loads(e.body)["title"])

Policy already exists


## 2. Applying the policy to a User and Role

In order for a policy to be applied to a user you must first add it to a role. This can be an existing role or a new one.

1. Create a new role or find an existing one to update - more details [HERE](https://support.lusid.com/knowledgebase/article/KA-01653/en-us)
2. Add the newly defined policy to the role.
3. Add user(s) to the new or existing role to implement the new access control.

### More information

For more information on FINBOURNE's Access and Identity Management tools and processes, please see the following set of tutorials and guides on the Knowledge Base: https://support.lusid.com/knowledgebase/article/KA-01647/en-us